In [1]:
!hadoop fs -head /public/hr_db/employees/part-m-00003

180	Winston	Taylor	WTAYLOR	650.507.9876	1998-01-24	SH_CLERK	3200.00	null	120	50
181	Jean	Fleaur	JFLEAUR	650.507.9877	1998-02-23	SH_CLERK	3100.00	null	120	50
182	Martha	Sullivan	MSULLIVA	650.507.9878	1999-06-21	SH_CLERK	2500.00	null	120	50
183	Girard	Geoni	GGEONI	650.507.9879	2000-02-03	SH_CLERK	2800.00	null	120	50
184	Nandita	Sarchand	NSARCHAN	650.509.1876	1996-01-27	SH_CLERK	4200.00	null	121	50
185	Alexis	Bull	ABULL	650.509.2876	1997-02-20	SH_CLERK	4100.00	null	121	50
186	Julia	Dellinger	JDELLING	650.509.3876	1998-06-24	SH_CLERK	3400.00	null	121	50
187	Anthony	Cabrio	ACABRIO	650.509.4876	1999-02-07	SH_CLERK	3000.00	null	121	50
188	Kelly	Chung	KCHUNG	650.505.1876	1997-06-14	SH_CLERK	3800.00	null	122	50
189	Jennifer	Dilly	JDILLY	650.505.2876	1997-08-13	SH_CLERK	3600.00	null	122	50
190	Timothy	Gates	TGATES	650.505.3876	1998-07-11	SH_CLERK	2900.00	null	122	50
191	Randall	Perkins	RPERKINS	650.505.4876	1999-12-19	SH_CLERK	2500.00	null	122	50
192	Sarah	Bell	SBELL	650.501.1876	1996-02-04	SH_C

In [2]:
import getpass as gp
from pyspark.sql import SparkSession, types as T, functions as F, Window as W

In [3]:
user = gp.getuser()
user

'itv005077'

In [4]:
spark = SparkSession.builder \
    .appName(f'{user}-Week-8-Assignment-3') \
    .master('yarn') \
    .config('spark.sql.warehouse.dir', f'/user/{user}/warehouse/') \
    .config('spark.sql.catalogImplementation', 'hive') \
    .enableHiveSupport() \
    .getOrCreate()

In [5]:
spark

In [6]:
schema = T.StructType([
    T.StructField('employee_id', T.IntegerType()),
    T.StructField('first_name', T.StringType()),
    T.StructField('last_name', T.StringType()),
    T.StructField('email', T.StringType()),
    T.StructField('phone_number', T.StringType()),
    T.StructField('hire_date', T.DateType()),
    T.StructField('job_id', T.StringType()),
    T.StructField('salary', T.FloatType()),
    T.StructField('commission_pct', T.FloatType()),
    T.StructField('manager_id', T.IntegerType()),
    T.StructField('department_id', T.IntegerType()),
])

In [7]:
df_employee = spark.read \
    .format('csv') \
    .option('delimiter', '\t') \
    .schema(schema) \
    .load('/public/hr_db/employees/')

In [8]:
df_employee.show(5)

+-----------+----------+---------+--------+------------+----------+--------+------+--------------+----------+-------------+
|employee_id|first_name|last_name|   email|phone_number| hire_date|  job_id|salary|commission_pct|manager_id|department_id|
+-----------+----------+---------+--------+------------+----------+--------+------+--------------+----------+-------------+
|        127|     James|   Landry| JLANDRY|650.124.1334|1999-01-14|ST_CLERK|2400.0|          null|       120|           50|
|        128|    Steven|   Markle| SMARKLE|650.124.1434|2000-03-08|ST_CLERK|2200.0|          null|       120|           50|
|        129|     Laura|   Bissot| LBISSOT|650.124.5234|1997-08-20|ST_CLERK|3300.0|          null|       121|           50|
|        130|     Mozhe| Atkinson|MATKINSO|650.124.6234|1997-10-30|ST_CLERK|2800.0|          null|       121|           50|
|        131|     James|   Marlow| JAMRLOW|650.124.7234|1997-02-16|ST_CLERK|2500.0|          null|       121|           50|
+-------

# 1. ROW_NUMBER

## Programatic Approach

In [9]:
# Order By No Partition By with Window Frame
window = W.orderBy(F.desc('employee_id')).rowsBetween(W.unboundedPreceding, W.currentRow)

In [10]:
df_employee \
.withColumn('row_num', F.row_number().over(window)) \
.show()

+-----------+----------+---------+--------+------------+----------+----------+-------+--------------+----------+-------------+-------+
|employee_id|first_name|last_name|   email|phone_number| hire_date|    job_id| salary|commission_pct|manager_id|department_id|row_num|
+-----------+----------+---------+--------+------------+----------+----------+-------+--------------+----------+-------------+-------+
|        206|   William|    Gietz|  WGIETZ|515.123.8181|1994-06-07|AC_ACCOUNT| 8300.0|          null|       205|          110|      1|
|        205|   Shelley|  Higgins|SHIGGINS|515.123.8080|1994-06-07|    AC_MGR|12000.0|          null|       101|          110|      2|
|        204|   Hermann|     Baer|   HBAER|515.123.8888|1994-06-07|    PR_REP|10000.0|          null|       101|           70|      3|
|        203|     Susan|   Mavris| SMAVRIS|515.123.7777|1994-06-07|    HR_REP| 6500.0|          null|       101|           40|      4|
|        202|       Pat|      Fay|    PFAY|603.123.6666

## SPARK SQL Approach

In [11]:
df_employee.createOrReplaceTempView('employees')

In [12]:
spark.sql('''
    SELECT *,
           row_number() over (order by employee_id desc rows between unbounded preceding and current row) as row_num
    FROM employees
''') \
.show()

+-----------+----------+---------+--------+------------+----------+----------+-------+--------------+----------+-------------+-------+
|employee_id|first_name|last_name|   email|phone_number| hire_date|    job_id| salary|commission_pct|manager_id|department_id|row_num|
+-----------+----------+---------+--------+------------+----------+----------+-------+--------------+----------+-------------+-------+
|        206|   William|    Gietz|  WGIETZ|515.123.8181|1994-06-07|AC_ACCOUNT| 8300.0|          null|       205|          110|      1|
|        205|   Shelley|  Higgins|SHIGGINS|515.123.8080|1994-06-07|    AC_MGR|12000.0|          null|       101|          110|      2|
|        204|   Hermann|     Baer|   HBAER|515.123.8888|1994-06-07|    PR_REP|10000.0|          null|       101|           70|      3|
|        203|     Susan|   Mavris| SMAVRIS|515.123.7777|1994-06-07|    HR_REP| 6500.0|          null|       101|           40|      4|
|        202|       Pat|      Fay|    PFAY|603.123.6666

# 2. RANK

## Programatic Approach

In [13]:
# Parttion By and Order By with Window Frame
window = W.partitionBy('department_id', 'job_id').orderBy(F.desc('salary')).rowsBetween(W.unboundedPreceding, W.currentRow)

In [14]:
df_employee \
.withColumn('rnk', F.rank().over(window)) \
.show()

# |        168|      Lisa|     Ozer|   LOZER|011.44.1343.929268|1997-03-11| SA_REP|11500.0|          0.25|       148|           80|  1|
# |        174|     Ellen|     Abel|   EABEL|011.44.1644.429267|1996-05-11| SA_REP|11000.0|           0.3|       149|           80|  2|
# |        162|     Clara|  Vishney|CVISHNEY|011.44.1346.129268|1997-11-11| SA_REP|10500.0|          0.25|       147|           80|  3|
# |        150|     Peter|   Tucker| PTUCKER|011.44.1344.129268|1997-01-30| SA_REP|10000.0|           0.3|       145|           80|  4|
# |        156|   Janette|     King|   JKING|011.44.1345.429268|1996-01-30| SA_REP|10000.0|          0.35|       146|           80|  4|
# |        169|  Harrison|    Bloom|  HBLOOM|011.44.1343.829268|1998-03-23| SA_REP|10000.0|           0.2|       148|           80|  4|
# |        170|    Tayler|      Fox|    TFOX|011.44.1343.729268|1998-01-24| SA_REP| 9600.0|           0.2|       148|           80|  7|<<<===
# |        151|     David|Bernstein|DBERNSTE|011.44.1344.345268|1997-03-24| SA_REP| 9500.0|          0.25|       145|           80|  8|
# |        157|   Patrick|    Sully|  PSULLY|011.44.1345.929268|1996-03-04| SA_REP| 9500.0|          0.35|       146|           80|  8|
# |        163|  Danielle|   Greene| DGREENE|011.44.1346.229268|1999-03-19| SA_REP| 9500.0|          0.15|       147|           80|  8|
# |        152|     Peter|     Hall|   PHALL|011.44.1344.478968|1997-08-20| SA_REP| 9000.0|          0.25|       145|           80| 11|<<<===
# |        158|     Allan|   McEwen| AMCEWEN|011.44.1345.829268|1996-08-01| SA_REP| 9000.0|          0.35|       146|           80| 11|

+-----------+----------+---------+--------+------------------+----------+-------+-------+--------------+----------+-------------+---+
|employee_id|first_name|last_name|   email|      phone_number| hire_date| job_id| salary|commission_pct|manager_id|department_id|rnk|
+-----------+----------+---------+--------+------------------+----------+-------+-------+--------------+----------+-------------+---+
|        100|    Steven|     King|   SKING|      515.123.4567|1987-06-17|AD_PRES|24000.0|          null|      null|           90|  1|
|        114|       Den| Raphaely|DRAPHEAL|      515.127.4561|1994-12-07| PU_MAN|11000.0|          null|       100|           30|  1|
|        204|   Hermann|     Baer|   HBAER|      515.123.8888|1994-06-07| PR_REP|10000.0|          null|       101|           70|  1|
|        121|      Adam|    Fripp|  AFRIPP|      650.123.2234|1997-04-10| ST_MAN| 8200.0|          null|       100|           50|  1|
|        120|   Matthew|    Weiss|  MWEISS|      650.123.1234|

## SPARK SQL Approach

In [15]:
df_employee.createOrReplaceTempView('employees')

In [16]:
spark.sql('''
    SELECT *,
           rank() over (partition by department_id, job_id order by salary desc rows between unbounded preceding and current row) as rnk
    FROM employees
''') \
.show()

# |        168|      Lisa|     Ozer|   LOZER|011.44.1343.929268|1997-03-11| SA_REP|11500.0|          0.25|       148|           80|  1|
# |        174|     Ellen|     Abel|   EABEL|011.44.1644.429267|1996-05-11| SA_REP|11000.0|           0.3|       149|           80|  2|
# |        162|     Clara|  Vishney|CVISHNEY|011.44.1346.129268|1997-11-11| SA_REP|10500.0|          0.25|       147|           80|  3|
# |        150|     Peter|   Tucker| PTUCKER|011.44.1344.129268|1997-01-30| SA_REP|10000.0|           0.3|       145|           80|  4|
# |        156|   Janette|     King|   JKING|011.44.1345.429268|1996-01-30| SA_REP|10000.0|          0.35|       146|           80|  4|
# |        169|  Harrison|    Bloom|  HBLOOM|011.44.1343.829268|1998-03-23| SA_REP|10000.0|           0.2|       148|           80|  4|
# |        170|    Tayler|      Fox|    TFOX|011.44.1343.729268|1998-01-24| SA_REP| 9600.0|           0.2|       148|           80|  7|<<<===
# |        151|     David|Bernstein|DBERNSTE|011.44.1344.345268|1997-03-24| SA_REP| 9500.0|          0.25|       145|           80|  8|
# |        157|   Patrick|    Sully|  PSULLY|011.44.1345.929268|1996-03-04| SA_REP| 9500.0|          0.35|       146|           80|  8|
# |        163|  Danielle|   Greene| DGREENE|011.44.1346.229268|1999-03-19| SA_REP| 9500.0|          0.15|       147|           80|  8|
# |        152|     Peter|     Hall|   PHALL|011.44.1344.478968|1997-08-20| SA_REP| 9000.0|          0.25|       145|           80| 11|<<<===
# |        158|     Allan|   McEwen| AMCEWEN|011.44.1345.829268|1996-08-01| SA_REP| 9000.0|          0.35|       146|           80| 11|

+-----------+----------+---------+--------+------------------+----------+-------+-------+--------------+----------+-------------+---+
|employee_id|first_name|last_name|   email|      phone_number| hire_date| job_id| salary|commission_pct|manager_id|department_id|rnk|
+-----------+----------+---------+--------+------------------+----------+-------+-------+--------------+----------+-------------+---+
|        100|    Steven|     King|   SKING|      515.123.4567|1987-06-17|AD_PRES|24000.0|          null|      null|           90|  1|
|        114|       Den| Raphaely|DRAPHEAL|      515.127.4561|1994-12-07| PU_MAN|11000.0|          null|       100|           30|  1|
|        204|   Hermann|     Baer|   HBAER|      515.123.8888|1994-06-07| PR_REP|10000.0|          null|       101|           70|  1|
|        121|      Adam|    Fripp|  AFRIPP|      650.123.2234|1997-04-10| ST_MAN| 8200.0|          null|       100|           50|  1|
|        120|   Matthew|    Weiss|  MWEISS|      650.123.1234|

# 3. DENSE_RANK()

## Programatic Approach

In [17]:
# Parttion By and Order By with Window Frame
window = W.partitionBy('department_id', 'job_id').orderBy(F.desc('salary')).rowsBetween(W.unboundedPreceding, W.currentRow)

In [18]:
df_employee \
.withColumn('dense_rnk', F.dense_rank().over(window))\
.show()

# |        168|      Lisa|     Ozer|   LOZER|011.44.1343.929268|1997-03-11| SA_REP|11500.0|          0.25|       148|           80|        1|
# |        174|     Ellen|     Abel|   EABEL|011.44.1644.429267|1996-05-11| SA_REP|11000.0|           0.3|       149|           80|        2|
# |        162|     Clara|  Vishney|CVISHNEY|011.44.1346.129268|1997-11-11| SA_REP|10500.0|          0.25|       147|           80|        3|
# |        150|     Peter|   Tucker| PTUCKER|011.44.1344.129268|1997-01-30| SA_REP|10000.0|           0.3|       145|           80|        4|
# |        156|   Janette|     King|   JKING|011.44.1345.429268|1996-01-30| SA_REP|10000.0|          0.35|       146|           80|        4|
# |        169|  Harrison|    Bloom|  HBLOOM|011.44.1343.829268|1998-03-23| SA_REP|10000.0|           0.2|       148|           80|        4|
# |        170|    Tayler|      Fox|    TFOX|011.44.1343.729268|1998-01-24| SA_REP| 9600.0|           0.2|       148|           80|        5|<<<====
# |        151|     David|Bernstein|DBERNSTE|011.44.1344.345268|1997-03-24| SA_REP| 9500.0|          0.25|       145|           80|        6|
# |        157|   Patrick|    Sully|  PSULLY|011.44.1345.929268|1996-03-04| SA_REP| 9500.0|          0.35|       146|           80|        6|
# |        163|  Danielle|   Greene| DGREENE|011.44.1346.229268|1999-03-19| SA_REP| 9500.0|          0.15|       147|           80|        6|
# |        152|     Peter|     Hall|   PHALL|011.44.1344.478968|1997-08-20| SA_REP| 9000.0|          0.25|       145|           80|        7|<<<===
# |        158|     Allan|   McEwen| AMCEWEN|011.44.1345.829268|1996-08-01| SA_REP| 9000.0|          0.35|       146|           80|        7|

+-----------+----------+---------+--------+------------------+----------+-------+-------+--------------+----------+-------------+---------+
|employee_id|first_name|last_name|   email|      phone_number| hire_date| job_id| salary|commission_pct|manager_id|department_id|dense_rnk|
+-----------+----------+---------+--------+------------------+----------+-------+-------+--------------+----------+-------------+---------+
|        100|    Steven|     King|   SKING|      515.123.4567|1987-06-17|AD_PRES|24000.0|          null|      null|           90|        1|
|        114|       Den| Raphaely|DRAPHEAL|      515.127.4561|1994-12-07| PU_MAN|11000.0|          null|       100|           30|        1|
|        204|   Hermann|     Baer|   HBAER|      515.123.8888|1994-06-07| PR_REP|10000.0|          null|       101|           70|        1|
|        121|      Adam|    Fripp|  AFRIPP|      650.123.2234|1997-04-10| ST_MAN| 8200.0|          null|       100|           50|        1|
|        120|   Matt

## SPARK SQL Approach

In [19]:
df_employee.createOrReplaceTempView('employees')

In [20]:
spark.sql('''
    SELECT *,
           dense_rank() over (partition by department_id, job_id order by salary desc rows between unbounded preceding and current row) as dense_rnk
    FROM employees
''') \
.show()

# |        168|      Lisa|     Ozer|   LOZER|011.44.1343.929268|1997-03-11| SA_REP|11500.0|          0.25|       148|           80|        1|
# |        174|     Ellen|     Abel|   EABEL|011.44.1644.429267|1996-05-11| SA_REP|11000.0|           0.3|       149|           80|        2|
# |        162|     Clara|  Vishney|CVISHNEY|011.44.1346.129268|1997-11-11| SA_REP|10500.0|          0.25|       147|           80|        3|
# |        150|     Peter|   Tucker| PTUCKER|011.44.1344.129268|1997-01-30| SA_REP|10000.0|           0.3|       145|           80|        4|
# |        156|   Janette|     King|   JKING|011.44.1345.429268|1996-01-30| SA_REP|10000.0|          0.35|       146|           80|        4|
# |        169|  Harrison|    Bloom|  HBLOOM|011.44.1343.829268|1998-03-23| SA_REP|10000.0|           0.2|       148|           80|        4|
# |        170|    Tayler|      Fox|    TFOX|011.44.1343.729268|1998-01-24| SA_REP| 9600.0|           0.2|       148|           80|        5|<<<===
# |        151|     David|Bernstein|DBERNSTE|011.44.1344.345268|1997-03-24| SA_REP| 9500.0|          0.25|       145|           80|        6|
# |        157|   Patrick|    Sully|  PSULLY|011.44.1345.929268|1996-03-04| SA_REP| 9500.0|          0.35|       146|           80|        6|
# |        163|  Danielle|   Greene| DGREENE|011.44.1346.229268|1999-03-19| SA_REP| 9500.0|          0.15|       147|           80|        6|
# |        152|     Peter|     Hall|   PHALL|011.44.1344.478968|1997-08-20| SA_REP| 9000.0|          0.25|       145|           80|        7|<<<===
# |        158|     Allan|   McEwen| AMCEWEN|011.44.1345.829268|1996-08-01| SA_REP| 9000.0|          0.35|       146|           80|        7|

+-----------+----------+---------+--------+------------------+----------+-------+-------+--------------+----------+-------------+---------+
|employee_id|first_name|last_name|   email|      phone_number| hire_date| job_id| salary|commission_pct|manager_id|department_id|dense_rnk|
+-----------+----------+---------+--------+------------------+----------+-------+-------+--------------+----------+-------------+---------+
|        100|    Steven|     King|   SKING|      515.123.4567|1987-06-17|AD_PRES|24000.0|          null|      null|           90|        1|
|        114|       Den| Raphaely|DRAPHEAL|      515.127.4561|1994-12-07| PU_MAN|11000.0|          null|       100|           30|        1|
|        204|   Hermann|     Baer|   HBAER|      515.123.8888|1994-06-07| PR_REP|10000.0|          null|       101|           70|        1|
|        121|      Adam|    Fripp|  AFRIPP|      650.123.2234|1997-04-10| ST_MAN| 8200.0|          null|       100|           50|        1|
|        120|   Matt

# 4. NTILE

## Programatic Approach

In [21]:
# Parttion By and Order By with Window Frame
window = W.partitionBy('job_id').orderBy('employee_id').rowsBetween(W.unboundedPreceding, W.currentRow)

In [22]:
df_employee\
.withColumn('nt', F.ntile(4).over(window)) \
.show()

# |        103|  Alexander|    Hunold| AHUNOLD|590.423.4567|1990-01-03|   IT_PROG| 9000.0|          null|       102|           60|  1|
# |        104|      Bruce|     Ernst|  BERNST|590.423.4568|1991-05-21|   IT_PROG| 6000.0|          null|       103|           60|  1|
# |        105|      David|    Austin| DAUSTIN|590.423.4569|1997-06-25|   IT_PROG| 4800.0|          null|       103|           60|  2|
# |        106|      Valli| Pataballa|VPATABAL|590.423.4560|1998-02-05|   IT_PROG| 4800.0|          null|       103|           60|  3|
# |        107|      Diana|   Lorentz|DLORENTZ|590.423.5567|1999-02-07|   IT_PROG| 4200.0|          null|       103|           60|  4|

+-----------+-----------+----------+--------+------------+----------+----------+-------+--------------+----------+-------------+---+
|employee_id| first_name| last_name|   email|phone_number| hire_date|    job_id| salary|commission_pct|manager_id|department_id| nt|
+-----------+-----------+----------+--------+------------+----------+----------+-------+--------------+----------+-------------+---+
|        109|     Daniel|    Faviet| DFAVIET|515.124.4169|1994-08-16|FI_ACCOUNT| 9000.0|          null|       108|          100|  1|
|        110|       John|      Chen|   JCHEN|515.124.4269|1997-09-28|FI_ACCOUNT| 8200.0|          null|       108|          100|  1|
|        111|     Ismael|   Sciarra|ISCIARRA|515.124.4369|1997-09-30|FI_ACCOUNT| 7700.0|          null|       108|          100|  2|
|        112|Jose Manuel|     Urman| JMURMAN|515.124.4469|1998-03-07|FI_ACCOUNT| 7800.0|          null|       108|          100|  3|
|        113|       Luis|      Popp|   LPOPP|515.124.4567|1999-12-07|

## SPARK SQL Approach

In [23]:
df_employee.createOrReplaceTempView('employee')

In [24]:
spark.sql('''
    SELECT *,
           ntile(4) over(partition by job_id order by employee_id rows between unbounded preceding and current row) as nt
    FROM employee
''') \
.show()

+-----------+-----------+----------+--------+------------+----------+----------+-------+--------------+----------+-------------+---+
|employee_id| first_name| last_name|   email|phone_number| hire_date|    job_id| salary|commission_pct|manager_id|department_id| nt|
+-----------+-----------+----------+--------+------------+----------+----------+-------+--------------+----------+-------------+---+
|        109|     Daniel|    Faviet| DFAVIET|515.124.4169|1994-08-16|FI_ACCOUNT| 9000.0|          null|       108|          100|  1|
|        110|       John|      Chen|   JCHEN|515.124.4269|1997-09-28|FI_ACCOUNT| 8200.0|          null|       108|          100|  1|
|        111|     Ismael|   Sciarra|ISCIARRA|515.124.4369|1997-09-30|FI_ACCOUNT| 7700.0|          null|       108|          100|  2|
|        112|Jose Manuel|     Urman| JMURMAN|515.124.4469|1998-03-07|FI_ACCOUNT| 7800.0|          null|       108|          100|  3|
|        113|       Luis|      Popp|   LPOPP|515.124.4567|1999-12-07|

# 5. LAG and LEAD

## Programatic Approach

In [25]:
# Parttion By and Order By with No Window Frame
# For lead and lag don't mention and frame rather define the offset and default value ``` F.lag('column', offset, default) ```
window = W.partitionBy('department_id').orderBy('salary')

In [26]:
df_employee \
.select('employee_id', 'department_id', 'salary') \
.withColumn('lag_salary', F.lag('salary', 1, 0).over(window)) \
.withColumn('lead_salary', F.lead('salary', 1, 0).over(window)) \
.show()

+-----------+-------------+-------+----------+-----------+
|employee_id|department_id| salary|lag_salary|lead_salary|
+-----------+-------------+-------+----------+-----------+
|        178|         null| 7000.0|       0.0|        0.0|
|        202|           20| 6000.0|       0.0|    13000.0|
|        201|           20|13000.0|    6000.0|        0.0|
|        203|           40| 6500.0|       0.0|        0.0|
|        113|          100| 6900.0|       0.0|     7700.0|
|        111|          100| 7700.0|    6900.0|     7800.0|
|        112|          100| 7800.0|    7700.0|     8200.0|
|        110|          100| 8200.0|    7800.0|     9000.0|
|        109|          100| 9000.0|    8200.0|    12000.0|
|        108|          100|12000.0|    9000.0|        0.0|
|        200|           10| 4400.0|       0.0|        0.0|
|        132|           50| 2100.0|       0.0|     2200.0|
|        128|           50| 2200.0|    2100.0|     2200.0|
|        136|           50| 2200.0|    2200.0|     2400.

## SPARK SQL Approach

In [27]:
df_employee.createOrReplaceTempView('employees')

In [28]:
spark.sql('''
    SELECT employee_id, department_id, salary,
           lag(salary, 1, 0) over (partition by department_id order by salary) as lag_salary,
           lead(salary, 1, 0) over (partition by department_id order by salary) as lead_salary
    FROM employees
''') \
.show()

+-----------+-------------+-------+----------+-----------+
|employee_id|department_id| salary|lag_salary|lead_salary|
+-----------+-------------+-------+----------+-----------+
|        178|         null| 7000.0|       0.0|        0.0|
|        202|           20| 6000.0|       0.0|    13000.0|
|        201|           20|13000.0|    6000.0|        0.0|
|        203|           40| 6500.0|       0.0|        0.0|
|        113|          100| 6900.0|       0.0|     7700.0|
|        111|          100| 7700.0|    6900.0|     7800.0|
|        112|          100| 7800.0|    7700.0|     8200.0|
|        110|          100| 8200.0|    7800.0|     9000.0|
|        109|          100| 9000.0|    8200.0|    12000.0|
|        108|          100|12000.0|    9000.0|        0.0|
|        200|           10| 4400.0|       0.0|        0.0|
|        132|           50| 2100.0|       0.0|     2200.0|
|        128|           50| 2200.0|    2100.0|     2200.0|
|        136|           50| 2200.0|    2200.0|     2400.

# 6. Runnig Total 

## Programatic Approach

In [29]:
# Parttion By and Order By with Window Frame
window = W.partitionBy('department_id').orderBy('salary').rowsBetween(W.unboundedPreceding, W.currentRow)

In [30]:
df_employee \
.select('employee_id', 'department_id', 'salary') \
.withColumn('running_total', F.sum('salary').over(window)) \
.show()

+-----------+-------------+-------+-------------+
|employee_id|department_id| salary|running_total|
+-----------+-------------+-------+-------------+
|        178|         null| 7000.0|       7000.0|
|        202|           20| 6000.0|       6000.0|
|        201|           20|13000.0|      19000.0|
|        203|           40| 6500.0|       6500.0|
|        113|          100| 6900.0|       6900.0|
|        111|          100| 7700.0|      14600.0|
|        112|          100| 7800.0|      22400.0|
|        110|          100| 8200.0|      30600.0|
|        109|          100| 9000.0|      39600.0|
|        108|          100|12000.0|      51600.0|
|        200|           10| 4400.0|       4400.0|
|        132|           50| 2100.0|       2100.0|
|        128|           50| 2200.0|       4300.0|
|        136|           50| 2200.0|       6500.0|
|        127|           50| 2400.0|       8900.0|
|        135|           50| 2400.0|      11300.0|
|        131|           50| 2500.0|      13800.0|


## SPARK SQL Approach

In [31]:
df_employee.createOrReplaceTempView('employees')

In [32]:
spark.sql('''
    SELECT employee_id, department_id, salary,
           sum(salary) over (partition by department_id order by salary rows between unbounded preceding and current row) as running_total
    FROM employees
''') \
.show()

+-----------+-------------+-------+-------------+
|employee_id|department_id| salary|running_total|
+-----------+-------------+-------+-------------+
|        178|         null| 7000.0|       7000.0|
|        202|           20| 6000.0|       6000.0|
|        201|           20|13000.0|      19000.0|
|        203|           40| 6500.0|       6500.0|
|        113|          100| 6900.0|       6900.0|
|        111|          100| 7700.0|      14600.0|
|        112|          100| 7800.0|      22400.0|
|        110|          100| 8200.0|      30600.0|
|        109|          100| 9000.0|      39600.0|
|        108|          100|12000.0|      51600.0|
|        200|           10| 4400.0|       4400.0|
|        132|           50| 2100.0|       2100.0|
|        128|           50| 2200.0|       4300.0|
|        136|           50| 2200.0|       6500.0|
|        127|           50| 2400.0|       8900.0|
|        135|           50| 2400.0|      11300.0|
|        131|           50| 2500.0|      13800.0|


# 7. Sum of a Group

## Programatic Approach

In [33]:
# Parttion By and No Order By with Window Frame
# For Group Total the Window Frame is UnBounded Preceding and Unbounded Following
window = W.partitionBy('department_id').rowsBetween(W.unboundedPreceding, W.unboundedFollowing)

In [34]:
df_employee \
.select('employee_id', 'department_id', 'salary') \
.withColumn('total_sum', F.sum('salary').over(window)) \
.show()

+-----------+-------------+-------+---------+
|employee_id|department_id| salary|total_sum|
+-----------+-------------+-------+---------+
|        178|         null| 7000.0|   7000.0|
|        201|           20|13000.0|  19000.0|
|        202|           20| 6000.0|  19000.0|
|        203|           40| 6500.0|   6500.0|
|        108|          100|12000.0|  51600.0|
|        109|          100| 9000.0|  51600.0|
|        110|          100| 8200.0|  51600.0|
|        111|          100| 7700.0|  51600.0|
|        112|          100| 7800.0|  51600.0|
|        113|          100| 6900.0|  51600.0|
|        200|           10| 4400.0|   4400.0|
|        127|           50| 2400.0| 156400.0|
|        128|           50| 2200.0| 156400.0|
|        129|           50| 3300.0| 156400.0|
|        130|           50| 2800.0| 156400.0|
|        131|           50| 2500.0| 156400.0|
|        132|           50| 2100.0| 156400.0|
|        133|           50| 3300.0| 156400.0|
|        134|           50| 2900.0

## SPARK SQL Approach

In [35]:
df_employee.createOrReplaceTempView('employees')

In [36]:
spark.sql('''
    SELECT employee_id, department_id, salary,
           sum(salary) over (partition by department_id rows between unbounded preceding and unbounded following) as total_sum
    FROM employees
''') \
.show()

+-----------+-------------+-------+---------+
|employee_id|department_id| salary|total_sum|
+-----------+-------------+-------+---------+
|        178|         null| 7000.0|   7000.0|
|        201|           20|13000.0|  19000.0|
|        202|           20| 6000.0|  19000.0|
|        203|           40| 6500.0|   6500.0|
|        108|          100|12000.0|  51600.0|
|        109|          100| 9000.0|  51600.0|
|        110|          100| 8200.0|  51600.0|
|        111|          100| 7700.0|  51600.0|
|        112|          100| 7800.0|  51600.0|
|        113|          100| 6900.0|  51600.0|
|        200|           10| 4400.0|   4400.0|
|        120|           50| 8000.0| 156400.0|
|        121|           50| 8200.0| 156400.0|
|        122|           50| 7900.0| 156400.0|
|        123|           50| 6500.0| 156400.0|
|        124|           50| 5800.0| 156400.0|
|        125|           50| 3200.0| 156400.0|
|        126|           50| 2700.0| 156400.0|
|        180|           50| 3200.0

# 8. FIRST Value

## Programatic Approach

In [37]:
# For Min value in a group order it ASC where the Window Frame is UnBounded Preceding and Current Row
window = W.partitionBy('department_id').orderBy('salary').rowsBetween(W.unboundedPreceding, W.currentRow)

In [38]:
df_employee \
.select('employee_id', 'department_id', 'salary') \
.withColumn('min_salary', F.min('salary').over(window)) \
.show()

+-----------+-------------+-------+----------+
|employee_id|department_id| salary|min_salary|
+-----------+-------------+-------+----------+
|        178|         null| 7000.0|    7000.0|
|        202|           20| 6000.0|    6000.0|
|        201|           20|13000.0|    6000.0|
|        203|           40| 6500.0|    6500.0|
|        113|          100| 6900.0|    6900.0|
|        111|          100| 7700.0|    6900.0|
|        112|          100| 7800.0|    6900.0|
|        110|          100| 8200.0|    6900.0|
|        109|          100| 9000.0|    6900.0|
|        108|          100|12000.0|    6900.0|
|        200|           10| 4400.0|    4400.0|
|        132|           50| 2100.0|    2100.0|
|        128|           50| 2200.0|    2100.0|
|        136|           50| 2200.0|    2100.0|
|        127|           50| 2400.0|    2100.0|
|        135|           50| 2400.0|    2100.0|
|        131|           50| 2500.0|    2100.0|
|        140|           50| 2500.0|    2100.0|
|        144|

## SPARK SQL Approach

In [39]:
df_employee.createOrReplaceTempView('employees')

In [40]:
spark.sql('''
    SELECT employee_id, department_id, salary,
           min(salary) over (partition by department_id order by salary rows between unbounded preceding and current row) as min_salary
    FROM employees
''') \
.show()

+-----------+-------------+-------+----------+
|employee_id|department_id| salary|min_salary|
+-----------+-------------+-------+----------+
|        178|         null| 7000.0|    7000.0|
|        202|           20| 6000.0|    6000.0|
|        201|           20|13000.0|    6000.0|
|        203|           40| 6500.0|    6500.0|
|        113|          100| 6900.0|    6900.0|
|        111|          100| 7700.0|    6900.0|
|        112|          100| 7800.0|    6900.0|
|        110|          100| 8200.0|    6900.0|
|        109|          100| 9000.0|    6900.0|
|        108|          100|12000.0|    6900.0|
|        200|           10| 4400.0|    4400.0|
|        132|           50| 2100.0|    2100.0|
|        128|           50| 2200.0|    2100.0|
|        136|           50| 2200.0|    2100.0|
|        127|           50| 2400.0|    2100.0|
|        135|           50| 2400.0|    2100.0|
|        131|           50| 2500.0|    2100.0|
|        140|           50| 2500.0|    2100.0|
|        144|

# 9. LAST Value

## Programatic Approach

In [41]:
# For Max value in a group order it DESC where the Window Frame is UnBounded Preceding and Current Row
window = W.partitionBy('department_id').orderBy(F.desc('salary')).rowsBetween(W.unboundedPreceding, W.currentRow)

In [42]:
df_employee \
.select('employee_id', 'department_id', 'salary') \
.withColumn('max_salary', F.max('salary').over(window)) \
.orderBy('department_id', 'salary') \
.show()

+-----------+-------------+-------+----------+
|employee_id|department_id| salary|max_salary|
+-----------+-------------+-------+----------+
|        178|         null| 7000.0|    7000.0|
|        200|           10| 4400.0|    4400.0|
|        202|           20| 6000.0|   13000.0|
|        201|           20|13000.0|   13000.0|
|        119|           30| 2500.0|   11000.0|
|        118|           30| 2600.0|   11000.0|
|        117|           30| 2800.0|   11000.0|
|        116|           30| 2900.0|   11000.0|
|        115|           30| 3100.0|   11000.0|
|        114|           30|11000.0|   11000.0|
|        203|           40| 6500.0|    6500.0|
|        132|           50| 2100.0|    8200.0|
|        136|           50| 2200.0|    8200.0|
|        128|           50| 2200.0|    8200.0|
|        135|           50| 2400.0|    8200.0|
|        127|           50| 2400.0|    8200.0|
|        140|           50| 2500.0|    8200.0|
|        182|           50| 2500.0|    8200.0|
|        131|

## SPARK SQL Approach

In [43]:
df_employee.createOrReplaceTempView('employees')

In [44]:
spark.sql('''
    SELECT employee_id, department_id, salary,
           max(salary) over (partition by department_id order by salary desc rows between unbounded preceding and current row) as max_salary
    FROM employees
    ORDER BY department_id, salary
''') \
.show()

+-----------+-------------+-------+----------+
|employee_id|department_id| salary|max_salary|
+-----------+-------------+-------+----------+
|        178|         null| 7000.0|    7000.0|
|        200|           10| 4400.0|    4400.0|
|        202|           20| 6000.0|   13000.0|
|        201|           20|13000.0|   13000.0|
|        119|           30| 2500.0|   11000.0|
|        118|           30| 2600.0|   11000.0|
|        117|           30| 2800.0|   11000.0|
|        116|           30| 2900.0|   11000.0|
|        115|           30| 3100.0|   11000.0|
|        114|           30|11000.0|   11000.0|
|        203|           40| 6500.0|    6500.0|
|        132|           50| 2100.0|    8200.0|
|        136|           50| 2200.0|    8200.0|
|        128|           50| 2200.0|    8200.0|
|        135|           50| 2400.0|    8200.0|
|        127|           50| 2400.0|    8200.0|
|        131|           50| 2500.0|    8200.0|
|        191|           50| 2500.0|    8200.0|
|        140|

In [45]:
spark.stop()